In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
import pickle

In [2]:
# Load preprocessed data
with open(
    "C:/Users/syafi/Desktop/syafiq-project/new classification task/model/saved_data/preprocessed_data.pkl",
    "rb",
) as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

# Display train and test sizes
print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

Train size: 1344719, Test size: 336180


In [3]:
# Define sizes for experiments
sample_sizes = [25, 50, 75, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 10000, 20000, 30000, 40000, 50000]
training_sizes = [20, 40, 60, 80, 200, 400, 600, 800, 2000, 4000, 6000, 8000, 16000, 24000, 32000, 40000]
testing_sizes = [5, 10, 15, 20, 50, 100, 150, 200, 500, 1000, 1500, 2000, 4000, 6000, 8000, 10000]

# Class labels for reporting
class_labels = [
    "Atherosclerosis",
    "Hypertension",
    "Cardiovascular Disease (CVD)",
    "Chronic Fatigue Syndrome (CFS)",
    "Respiratory Disease (COPD or Asthma)",
    "Stress-related Disorders",
    "Arrhythmias",
    "Healthy",
    "Autonomic Dysfunction",
    "Diabetes",
    "Anaemia",
]

In [4]:
def evaluate_model(model, X_test_subset, y_test_subset, class_labels):
    """
    Evaluate the model and return classification metrics.
    """
    # Predictions
    predictions = model.predict(X_test_subset)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_subset, predictions)

    # Generate dynamic class labels
    unique_classes = np.unique(y_test_subset)
    dynamic_labels = [class_labels[i] for i in unique_classes]

    # Generate classification report
    report = classification_report(
        y_test_subset,
        predictions,
        target_names=dynamic_labels,
        labels=unique_classes,
        zero_division=0,
        output_dict=True,
    )

    return accuracy, report, predictions, unique_classes, dynamic_labels

In [5]:
def flatten_classification_report(report, sample_size, train_size, test_size, accuracy):
    """
    Flatten the classification report for easier analysis and saving.
    """
    flat_report = {
        f"{label}_{metric}": value
        for label, metrics in report.items()
        if isinstance(metrics, dict)  # Exclude keys like 'accuracy'
        for metric, value in metrics.items()
    }
    flat_report.update(
        {
            "sample_size": sample_size,
            "train_size": train_size,
            "test_size": test_size,
            "accuracy": accuracy,
        }
    )
    return flat_report

In [6]:
def expand_test_set(X_test, y_test, repeat_factor):
    """
    Expand the test set by repeating each sample multiple times.

    Args:
        X_test (ndarray): Original test features.
        y_test (ndarray): Original test labels.
        repeat_factor (int): Number of times to repeat each test sample.

    Returns:
        Expanded test features and labels.
    """
    X_test_expanded = np.repeat(X_test, repeats=repeat_factor, axis=0)
    y_test_expanded = np.repeat(y_test, repeats=repeat_factor)
    return X_test_expanded, y_test_expanded

In [7]:
# Container for results
results = []
best_accuracy = 0
best_model = None
best_sample_size = 0

# Iterate through sample, training, and testing sizes
for sample_size, train_size, test_size in zip(
    sample_sizes, training_sizes, testing_sizes
):
    # Select subset of training and testing data
    X_train_subset, y_train_subset = X_train[:train_size], y_train[:train_size]
    X_test_subset, y_test_subset = X_test[:test_size], y_test[:test_size]

    # Expand the test set for alignment
    repeat_factor = 11  # Adjust to ensure total support aligns with expectations
    X_test_expanded, y_test_expanded = expand_test_set(
        X_test_subset, y_test_subset, repeat_factor
    )

    # Fit the Naive Bayes model
    nb_model = GaussianNB()
    nb_model.fit(X_train_subset, y_train_subset)

    # Evaluate the model
    accuracy, report, predictions, unique_classes, dynamic_labels = evaluate_model(
        nb_model, X_test_expanded, y_test_expanded, class_labels
    )

    # Print results for the current iteration
    print(f"\nNaive Bayes Sample size {sample_size} - Accuracy: {accuracy:.4f}")
    print(
        classification_report(
            y_test_expanded,
            predictions,
            target_names=dynamic_labels,
            labels=unique_classes,
            zero_division=0,
        )
    )

    # Flatten report and store results
    flat_report = flatten_classification_report(
        report, sample_size, train_size, test_size, accuracy
    )
    results.append(flat_report)

    # Update the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = nb_model
        best_sample_size = sample_size
        print(
            f"New best model found for sample size {sample_size} with accuracy {accuracy:.4f}"
        )


Naive Bayes Sample size 25 - Accuracy: 0.0000
                                precision    recall  f1-score   support

Chronic Fatigue Syndrome (CFS)       0.00      0.00      0.00      11.0
      Stress-related Disorders       0.00      0.00      0.00      11.0
                       Healthy       0.00      0.00      0.00      11.0
         Autonomic Dysfunction       0.00      0.00      0.00      11.0
                      Diabetes       0.00      0.00      0.00      11.0

                     micro avg       0.00      0.00      0.00      55.0
                     macro avg       0.00      0.00      0.00      55.0
                  weighted avg       0.00      0.00      0.00      55.0


Naive Bayes Sample size 50 - Accuracy: 0.1000
                                precision    recall  f1-score   support

                  Hypertension       0.00      0.00      0.00        11
Chronic Fatigue Syndrome (CFS)       0.00      0.00      0.00        11
      Stress-related Disorders       0

In [ ]:
# Save the best model
if best_model is not None:
    best_model_file = f"best_nb_model_sample_size_{best_sample_size}.pkl"
    with open(best_model_file, "wb") as model_file:
        pickle.dump(best_model, model_file)
    print(f"\nBest model saved as {best_model_file} with accuracy {best_accuracy:.4f}")

In [ ]:
# Save results to a CSV file for further analysis
results_df = pd.DataFrame(results)
results_df.to_csv("naive_bayes_results.csv", index=False)
print("\nResults saved to 'naive_bayes_results.csv'")

In [ ]:
# Display train and test set details
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")
print("\nClass Distribution in Test Set:")
print(y_test.value_counts())